In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA.xlsx')
data.head()

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 12)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    # part_of_model = dm[dm.index != user].reset_index()
    part_of_model = data_for_model.copy()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model.loc[part_of_model['nama'] == user, 'rating'] = 0
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

def tf_idf_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(md["desc"])
    model = X[:len(part_of_model)]
    query = X[len(part_of_model):]
    return model, query

In [16]:
from geopy.distance import great_circle
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = great_circle(loc1, loc2).km
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [17]:
def get_numtext_content(query_data_todict, part_of_model):
    tfidf_model, tfidf_query = tf_idf_vector(part_of_model, query_data_todict)
    cos_sim = cosine_similarity(tfidf_query, tfidf_model)
    data_sim = cos_sim.reshape(-1,1)
    distance = calculate_distance(query_data_todict, part_of_model)
    return data_sim, distance

In [18]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','distance', 'sim']].to_numpy()
    scaler = StandardScaler().fit(tmp_data)
    scaled_data = scaler.transform(tmp_data)
    sim_num = cosine_similarity(scaled_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    return sim_num

In [19]:
def value_of_mua(query_data_todict, part_of_model):
    sim, distance = get_numtext_content(query_data_todict, part_of_model)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)

    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(part_of_model["sim_num"][j])
        value_mua[i] = np.mean(tmp_val)
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

In [20]:
# nama_mua = part_of_model["nama_MUA"].unique()
# value_mua = np.empty(len(nama_mua), dtype=float)

# for i in range(len(nama_mua)):
#     tmp_val = []
#     for j in range(len(part_of_model.index)):
#         if nama_mua[i] == part_of_model.nama_MUA[j]:
#             tmp_val.append(part_of_model["sim_num"][j])
#     value_mua[i] = np.mean(tmp_val)


In [21]:
# query_data, input_data, data_ = get_content_values(data_content, 1)
# data_ = get_num_values(query_data, data_)
# pv_gcv = pd.pivot_table(data_, columns='nama_MUA', values='sim', aggfunc='mean')
# data_hyb = data_.copy()
# data_hyb = data_hyb[['sim', 'sim_num']]
# data_hyb ['total_sim'] = data_hyb.mean(axis=1)
# data_hyb ['nama_MUA'] = data_['nama_MUA']
# data_hyb ['rating'] = data_['rating'].astype(float)
# data_hyb ['nama'] = data_['nama']
# data_hyb = data_hyb[data_hyb.nama != query_data.nama[0]]
# data_hyb = data_hyb.sort_values(by=['total_sim'], ascending=False, ignore_index=True)
# pv_dhb = pd.pivot_table(data_hyb, columns='nama_MUA', values='total_sim', aggfunc='mean')
# list_sim = pv_dhb.values.tolist()[0]
# pv_dhb2 = pd.pivot_table(data_hyb, columns='nama_MUA', index='nama', values='rating', fill_value=0)
# list_nama = pv_dhb2.index.tolist()

## FIND USER

In [22]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [30]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre
import statistics 

In [31]:
#chosen user for testing
mse_hy = []
mae_hy = []

for u in range(len(selected_user)):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j] * 0.5
    
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
    
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        if weight_user[i] != 0.0:
            mean_weight[i] = statistics.harmonic_mean([weight_user[i],  abs(weight_content[i])])
        else:
            mean_weight[i] = weight_content[i] * 0.05
    final_res['mean_'] = mean_weight
    final_res = final_res.sort_values(by=['mean_'], ignore_index=True, ascending=False)
    get_user = final_res.groupby("nama").mean_.agg(["mean"])
    list_nama = get_user.index[:10].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    value_rating = np.zeros(len(col_model), dtype=float)
    for i in range(len(pivot_.index)):
        for j in range(len(col_model)):
            if value_rating[j] == 0.0:
                if col_model[j] == pivot_.index[i]:
                    value_rating[j] = (pivot_["mean"][i] * pivot_['weight'][i])
            else:
                continue
            
    # final_model = part_of_model.copy()
    # final_model = final_model.sort_values(by=["sim_num"], ascending=False, ignore_index=True)
    # highest_user = final_model["nama"][0]
    # pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    # col_model = pivot.columns.tolist()
    # val_model = np.array(pivot.values.tolist(), dtype=float)
    # nama_user = np.array(pivot.index.tolist())
    # for i in range(len(val_model)):
    #     for j in range(len(val_model[i])):
    #         if val_model[i][j] == 0.0:
    #             val_model[i][j] = value_mua[j]
#     pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#     val_query = pivot_query.values.tolist()
#     col_query = pivot_query.columns.tolist()
#     feature = np.zeros(len(col_model), dtype=float)
#     for i in range(len(col_model)):
#         for j in range(len(col_query)):
#             if col_model[i] == col_query[j]:
#                 feature[i] = val_query[0][j]
#                 break
#             else:
#                 feature[i] = value_mua[j]
#     knn = NearestNeighbors(metric='cosine', algorithm='brute')
#     knn.fit(val_model)
#     distances, indices = knn.kneighbors([feature], n_neighbors=10)
#     list_nama = []
#     list_nama = []
#     for ln in indices[0]:
#          list_nama.append(nama_user[ln])
#     pivot = part_of_model[part_of_model.nama.isin(list_nama)].reset_index()
#     pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
#     pivot_['cos'] = pivot.groupby("nama_MUA").sim_num.agg(['mean'])
#     value_rating = np.zeros(len(col_model), dtype=float)
#     for i in range(len(pivot_.index)):
#         for j in range(len(col_model)):
#             if value_rating[j] == 0.0:
#                 if col_model[j] == pivot_.index[i]:
#                     value_rating[j] = (pivot_["mean"][i] * pivot_["cos"][i])
#             else:
#                 continue
#     # mean_value_rating = value_rating.mean(axis=0)
    tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
    mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
    pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_label = pivot_label.values.tolist()
    col_label = pivot_label.columns.tolist()
    final_label = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_label)):
            if col_model[i] == col_label[j]:
                final_label[i] = val_label[0][j]
                break
    mae = mean_absolute_error(value_rating, final_label)
    mse = mean_squared_error(value_rating, final_label, squared=False)
    mae_hy.append(mae)
    mse_hy.append(mse)
    
mean_mae_hy = np.mean(mae_hy)
mean_mse_hy = np.mean(mse_hy)

In [32]:
mua_recs[mua_recs.score > 0.0]

,nama_mua,score
0,Dinda Wijaya Make Up,0.421541
1,Liyan Make Up Jember,0.387668
2,Laura Make Up & Bridal,0.211434
3,Bonita MUA Jember,0.140450
4,Larasnada Make Up,0.122528
5,Ulfa Beauty MUA,0.095354
6,Griya Putri Make Up,0.066392
7,Ardie Make Up & Salon,0.022491
8,Khaliza Make Up,0.014198
9,Novi Make Up,0.010365


## TESTING

Highest MAE

In [33]:
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user.index(73), query_user[selected_user.index(73)])
nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
col_model = pivot.columns.tolist()
val_model = np.array(pivot.values.tolist(), dtype=float)
nama_user = np.array(pivot.index.tolist())
pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
val_query = pivot_query.values.tolist()
col_query = pivot_query.columns.tolist()
feature = np.zeros(len(col_model), dtype=float)
for i in range(len(col_model)):
    for j in range(len(col_query)):
        if col_model[i] == col_query[j]:
            feature[i] = val_query[0][j]
            break
cos_sim_user = cosine_similarity([feature], val_model)

weight_user = np.empty(len(part_of_model), dtype=float)
for i in range(len(part_of_model)):
    for j in range(len(nama_user)):
        if part_of_model["nama"][i] == nama_user[j]:
            weight_user[i] = cos_sim_user[0][j]
    
weight_content = np.empty(len(part_of_model), dtype=float)
for i in range(len(part_of_model)):
    for j in range(len(nama_mua)):
        if part_of_model["nama_MUA"][i] == nama_mua[j]:
            weight_content[i] = value_mua[j]
    
final_res = part_of_model.copy()
final_res = final_res[['nama', 'nama_MUA']]
final_res['user'] = weight_user
final_res['content'] = weight_content

mean_weight = np.zeros(len(part_of_model), dtype=float)
for i in range(len(part_of_model)):
    if weight_user[i] != 0.0:
        mean_weight[i] = statistics.harmonic_mean([weight_user[i],  abs(weight_content[i])])
    else:
        mean_weight[i] = weight_content[i] * 0.05
final_res['mean_'] = mean_weight
final_res = final_res.sort_values(by=['mean_'], ignore_index=True, ascending=False)
get_user = final_res.groupby("nama").mean_.agg(["mean"])
list_nama = get_user.index[:10].tolist()
pivot = part_of_model.copy()
pivot['mean_'] = mean_weight
pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
pivot_['score'] = pivot_["mean"] * pivot_['weight']
pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)

In [34]:
pivot_

,count,mean,nama_MUA,weight,score,desc
0,2,5.000000,Dinda Wijaya Make Up,0.034039,0.170195,[LT Pro smooth corrector cream foundation - na...
1,2,4.000000,Bonita MUA Jember,0.015058,0.060233,[la tulipe cover foundation - suntan (sawo mat...
2,3,5.000000,Rias Manten Mariyam,0.001426,0.007128,[la tulipe cover foundation - natural (putih) ...
3,2,5.000000,Abha Beauty Make Up Artist,0.001254,0.006268,[LT Pro smooth corrector cream foundation - wa...
4,3,5.000000,Sanggar Rias Nining,0.001165,0.005825,[la tulipe cover foundation - natural (putih) ...
5,3,5.000000,Khaliza Make Up,0.000898,0.004489,[la tulipe cover foundation - natural (putih) ...
6,3,3.000000,Sanggar Rias dan Salon Esthie,0.001403,0.004210,[LT Pro smooth corrector cream foundation - wa...
7,5,4.600000,Yulia Salon & Bridal,0.000706,0.003246,[Maybelline Fit Me Matte + Poreless Liquid Fou...
8,3,4.000000,Novi Make Up,0.000648,0.002592,[la tulipe cover foundation - natural (putih) ...
9,4,3.500000,Ardie Make Up & Salon,0.000595,0.002082,[la tulipe cover foundation - natural (putih) ...


Lowest MAE

In [35]:
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user.index(45), query_user[selected_user.index(45)])
nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
col_model = pivot.columns.tolist()
val_model = np.array(pivot.values.tolist(), dtype=float)
nama_user = np.array(pivot.index.tolist())
pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
val_query = pivot_query.values.tolist()
col_query = pivot_query.columns.tolist()
feature = np.zeros(len(col_model), dtype=float)
for i in range(len(col_model)):
    for j in range(len(col_query)):
        if col_model[i] == col_query[j]:
            feature[i] = val_query[0][j]
            break
cos_sim_user = cosine_similarity([feature], val_model)

weight_user = np.empty(len(part_of_model), dtype=float)
for i in range(len(part_of_model)):
    for j in range(len(nama_user)):
        if part_of_model["nama"][i] == nama_user[j]:
            weight_user[i] = cos_sim_user[0][j]
    
weight_content = np.empty(len(part_of_model), dtype=float)
for i in range(len(part_of_model)):
    for j in range(len(nama_mua)):
        if part_of_model["nama_MUA"][i] == nama_mua[j]:
            weight_content[i] = value_mua[j]
    
final_res = part_of_model.copy()
final_res = final_res[['nama', 'nama_MUA']]
final_res['user'] = weight_user
final_res['content'] = weight_content

mean_weight = np.zeros(len(part_of_model), dtype=float)
for i in range(len(part_of_model)):
    if weight_user[i] != 0.0:
        mean_weight[i] = statistics.harmonic_mean([weight_user[i],  abs(weight_content[i])])
    else:
        mean_weight[i] = weight_content[i] * 0.05
final_res['mean_'] = mean_weight
final_res = final_res.sort_values(by=['mean_'], ignore_index=True, ascending=False)
get_user = final_res.groupby("nama").mean_.agg(["mean"])
list_nama = get_user.index[:10].tolist()
pivot = part_of_model.copy()
pivot['mean_'] = mean_weight
pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
pivot_['score'] = pivot_["mean"] * pivot_['weight']
pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)

In [36]:
pivot_

,count,mean,nama_MUA,weight,score,desc
0,3,5.000000,Khaliza Make Up,0.031834,0.159171,[la tulipe cover foundation - natural (putih) ...
1,3,4.000000,Novi Make Up,0.037358,0.149433,[la tulipe cover foundation - natural (putih) ...
2,2,4.000000,Bonita MUA Jember,0.025740,0.102959,[la tulipe cover foundation - suntan (sawo mat...
3,5,4.600000,Yulia Salon & Bridal,0.019402,0.089251,[Maybelline Fit Me Matte + Poreless Liquid Fou...
4,2,5.000000,Dinda Wijaya Make Up,0.010086,0.050432,[LT Pro smooth corrector cream foundation - na...
5,3,3.666667,Larasnada Make Up,0.011078,0.040619,[la tulipe cover foundation - suntan (sawo mat...
6,5,2.400000,Salon Fatah,0.010402,0.024965,[la tulipe cover foundation - natural (putih) ...
7,4,1.500000,Ardie Make Up & Salon,0.014822,0.022232,[la tulipe cover foundation - natural (putih) ...
8,2,5.000000,Aulia Rias Pengantin,0.001495,0.007477,[la tulipe cover foundation - suntan (sawo mat...
9,3,4.000000,Griya Putri Make Up,0.001681,0.006723,[Wardah Everyday Luminous Liquid Foundation - ...


In [37]:
# zz = pivot[['no', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
# zz['desc'] = zz['produk_makeup'] + ' - ' + zz['shade'] + ' - ' + zz['skin_color'] + ' - ' + zz['skin_undertone']
# zz = zz.groupby('no').agg({'desc': lambda x: list(x)}).reset_index()

In [38]:
mean_mae_hy, mean_mse_hy 

(0.3624409131528424, 1.199335850781883)

## COLLABORATIVE FILTERING

In [39]:
mse_col = []
mae_col = []

for u in range(len(selected_user)):
    _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]

    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user

    final_res = final_res.sort_values(by=['user'], ignore_index=True, ascending=False)
    get_user = final_res.groupby("nama").user.agg(["mean"])
    list_nama = get_user.index[:10].tolist()
    pivot = part_of_model.copy()
    pivot['user'] = weight_user
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['weight'] = pivot.groupby("nama_MUA").user.agg(['mean'])
    value_rating = np.zeros(len(col_model), dtype=float)
    for i in range(len(pivot_.index)):
        for j in range(len(col_model)):
            if value_rating[j] == 0.0:
                if col_model[j] == pivot_.index[i]:
                    value_rating[j] = (pivot_["mean"][i] * pivot_['weight'][i])
            else:
                continue
#     pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#     col_model = pivot.columns.tolist()
#     val_model = np.array(pivot.values.tolist(), dtype=float)
#     nama_user = np.array(pivot.index.tolist())
#     pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#     val_query = pivot_query.values.tolist()
#     col_query = pivot_query.columns.tolist()
#     feature = np.zeros(len(col_model), dtype=float)
#     for i in range(len(col_model)):
#         for j in range(len(col_query)):
#             if col_model[i] == col_query[j]:
#                 feature[i] = val_query[0][j]
#                 break
#     knn = NearestNeighbors(metric='cosine', algorithm='brute')
#     knn.fit(val_model)
#     distances, indices = knn.kneighbors([feature], n_neighbors=10)
#     list_nama = []
#     for ln in indices[0]:
#          list_nama.append(nama_user[ln])
#     pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
#     tmp_cos = np.empty(len(pivot), dtype=float)
#     for i in range(len(pivot.nama)):
#         for j in range(len(list_nama)):
#             if pivot.nama[i] == list_nama[j]:
#                 tmp_cos[i] = distances[0][j]
#     pivot['cos'] = tmp_cos
#     pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
#     pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
#     value_rating = np.zeros(len(col_model), dtype=float)
#     for i in range(len(pivot_.index)):
#         for j in range(len(col_model)):
#             if value_rating[j] == 0.0:
#                 if col_model[j] == pivot_.index[i]:
#                     value_rating[j] = (pivot_["mean"][i] * pivot_["cos"][i])
#             else:
#                 continue

# mse_col = []
# mae_col = []

# for u in range(len(selected_user)):
#     _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
#     pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#     col_model = pivot.columns.tolist()
#     val_model = np.array(pivot.values.tolist(), dtype=float)
#     nama_user = np.array(pivot.index.tolist())
#     pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#     val_query = pivot_query.values.tolist()
#     col_query = pivot_query.columns.tolist()
#     feature = np.zeros(len(col_model), dtype=float)
#     for i in range(len(col_model)):
#         for j in range(len(col_query)):
#             if col_model[i] == col_query[j]:
#                 feature[i] = val_query[0][j]
#                 break
#     knn = NearestNeighbors(metric='cosine', algorithm='brute')
#     knn.fit(val_model)
#     distances, indices = knn.kneighbors([feature], n_neighbors=10)
#     list_nama = []
#     for ln in indices[0]:
#          list_nama.append(nama_user[ln])
#     pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
#     tmp_cos = np.empty(len(pivot), dtype=float)
#     for i in range(len(pivot.nama)):
#         for j in range(len(list_nama)):
#             if pivot.nama[i] == list_nama[j]:
#                 tmp_cos[i] = distances[0][j]
#     pivot['cos'] = tmp_cos
#     pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
#     pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
#     value_rating = np.zeros(len(col_model), dtype=float)
#     for i in range(len(pivot_.index)):
#         for j in range(len(col_model)):
#             if value_rating[j] == 0.0:
#                 if col_model[j] == pivot_.index[i]:
#                     value_rating[j] = (pivot_["mean"][i] * pivot_['cos'][i])/pivot_['count'][i]
#             else:
#                 continue

    tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
    mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
    pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_label = pivot_label.values.tolist()
    col_label = pivot_label.columns.tolist()
    final_label = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_label)):
            if col_model[i] == col_label[j]:
                final_label[i] = val_label[0][j]
                break
    mae = mean_absolute_error(value_rating, final_label)
    mse = mean_squared_error(value_rating, final_label, squared=False)
    mae_col.append(mae)
    mse_col.append(mse)
    
mean_mae_col = np.mean(mae_col)
mean_mse_col = np.mean(mse_col)

In [40]:
mua_recs[mua_recs.score > 0.0]

,nama_mua,score
0,Dinda Wijaya Make Up,2.378807
1,Laura Make Up & Bridal,2.134146
2,Bonita MUA Jember,1.903045
3,Ulfa Beauty MUA,1.707317
4,Liyan Make Up Jember,1.707317
5,Larasnada Make Up,1.684652
6,Ardie Make Up & Salon,0.832582
7,Griya Putri Make Up,0.569106


In [41]:
mean_mae_col, mean_mse_col

(0.4159207504617984, 1.1656067553986595)

## CONTENT BASED FILTERING

In [46]:
mae_con = []
mse_con = []

for u in range(len(selected_user)):
    _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[u], query_user[u])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    get_user = part_of_model.groupby("nama").sim_num.agg(["mean"])
    get_user = get_user.sort_values(by=["mean"], ascending=False)
    list_nama = get_user.index[:10].tolist()
    pivot = part_of_model[part_of_model.nama.isin(list_nama)].reset_index()
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['cos'] = pivot.groupby("nama_MUA").sim_num.agg(['mean'])
    value_rating = np.zeros(len(col_model), dtype=float)
    for i in range(len(pivot_.index)):
        for j in range(len(col_model)):
            if value_rating[j] == 0.0:
                if col_model[j] == pivot_.index[i]:
                    value_rating[j] = (pivot_["mean"][i] * pivot_["cos"][i])
            else:
                continue
    tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
    mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
    pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_label = pivot_label.values.tolist()
    col_label = pivot_label.columns.tolist()
    final_label = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_label)):
            if col_model[i] == col_label[j]:
                final_label[i] = val_label[0][j]
                break
    mae = mean_absolute_error(value_rating, final_label)
    mse = mean_squared_error(value_rating, final_label, squared=False)
    mae_con.append(mae)
    mse_con.append(mse)
    
mean_mae_con = np.mean(mae_con)
mean_mse_con = np.mean(mse_con)

In [47]:
mua_recs[mua_recs.score > 0.0]

,nama_mua,score
0,Sanggar Rias Cantik Tina,0.250561
1,Griya Citra Ayu,0.229207
2,Aulia Rias Pengantin,0.212101
3,KL Make Up Private Studio,0.207662
4,Yulia MUA Wedding,0.192432
5,Aira Salon,0.143160
6,Wedding Gallery Make Over,0.130565
7,Salon Sylvia,0.128261
8,Griya Putri Make Up,0.125542
9,Winda Wijaya MUA & Beauty Studio,0.123000


In [48]:
mean_mae_con

0.3834089556112554

In [49]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mean_mae_col, mean_mae_con, mean_mae_hy],
    'MSE' : [mean_mse_col, mean_mse_con, mean_mse_hy]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE', 'MSE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,MSE
0,Hybrid Filtering,0.362441,1.199336
1,Content Based Filtering,0.383409,1.209545
2,Collaborative Filtering,0.415921,1.165607
